In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')

#mpl.rc('figure', figsize=(10,8))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json
import re
import os

#import copy

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *
from ur import *

from data import CAPData


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Import data

In [ ]:
data_folder='./Data/SP-2021_01_12-Q394_fmaskedCAP_data'

listFiles = os.listdir(data_folder)

In [ ]:
capData=CAPData(data_folder, listFiles, begin_ind=395, end_ind=2000)

In [ ]:
t=capData.t

broadband1=capData.get_signal_by_name('broadband_noise')
broadband2=capData.get_signal_by_name('2_broadband_noise')
broadband3=capData.get_signal_by_name('3_broadband_noise')

broadband_avg=(broadband1+broadband2+broadband3)/3



nomasker1=capData.get_signal_by_name('nomasker')
nomasker2=capData.get_signal_by_name('2_nomasker')
nomasker3=capData.get_signal_by_name('3_nomasker')

nomasker_avg=(nomasker1+nomasker2+nomasker3)/3

pl.plot(t, broadband1)
pl.plot(t, broadband2)
pl.plot(t, broadband3)
pl.plot(t, broadband_avg, label='avg')


pl.plot(t, nomasker1)

pl.plot(t, nomasker2)

pl.plot(t, nomasker3)
pl.plot(t, nomasker_avg, label='avg')

pl.legend()



In [ ]:
#window

t0=5.7e-3
t1=10e-3
ind0=int(t0*48828)

ind0=int(t0*48828)
ind1=int(t1*48828)

win0=sg.tukey(ind1-ind0, alpha=0.4)

win=np.zeros_like(broadband_avg)
win[ind0:ind1]=win0

pl.figure()
pl.plot(t*1e3, broadband_avg, label='avg')
pl.plot(t*1e3, win*np.amax(broadband_avg))

pl.plot(t*1e3, 0.5*broadband_avg*win, label='avg')
pl.show()


In [ ]:
def process_signal(sig, cumsum=True):
    sig2=sig*win
    
    t0=3e-3
    t1=13e-3

    ind0=int(t0*48828)
    ind1=int(t1*48828)
    
    dim = len(np.shape(sig2))
    if dim ==1:
        sig2=sig2[ind0:ind1]
        if cumsum:
            sig2=np.cumsum(sig2)
            sig2[0:-50]*=sg.tukey(len(sig2)-50, 0.3)
            sig2[-50:]=0
    else:
        sig2=sig2[:, ind0:ind1]
        if cumsum:
            sig2=np.cumsum(sig2)
            
            
    
    t=np.linspace(t0, t1, ind1-ind0)
    
    return t, sig2
    
    

    

In [ ]:
t2, broadband_proc=process_signal(broadband_avg, cumsum=True)
_, nomasker_proc=process_signal(nomasker_avg)

pl.plot(t2*1e3, nomasker_proc-broadband_proc)

**Estimation ur**  
Depends on what is the focus


In [ ]:
sig=capData.get_signal_by_name('2_notch3000_bw1700_55dB')
_, sig2=process_signal(sig)

#sigbis=capData.get_signal_by_name('3_notch3000_bw1700_50dB')
#_, sig2bis=process_signal(sigbis)



'''pl.plot(t*1e3, sig)

pl.plot(t*1e3, broadband_avg)
pl.plot(t*1e3, sig-broadband_avg)
'''
pl.plot(t2*1e3, sig2-broadband_proc)

#pl.plot(t2*1e3, sig2bis-broadband_proc)


ur0=sig2-broadband_proc
ur0=np.roll(ur0, -100)
pl.plot(t2*1e3, ur0)

In [ ]:
masked_sig=nomasker_proc-broadband_proc
masked_sig_fft=np.fft.rfft(masked_sig)
ur0_fft=np.fft.rfft(ur0)
E0_fft=masked_sig_fft/(ur0_fft+1e-2)
E0=np.fft.irfft(E0_fft)

In [ ]:
pl.plot(t2*1e3, E0)